Ensure you are using the preinstalled Jupyter kernel `amcinsights`

# Tenant Provisioning Service

Each customer is defined by a single AMC Instance

#### -----------------------
## Global Configs
Run this cell to initialize the notebook interface

In [ ]:
import os
import time

import boto3
import re 
import json
from dotenv import load_dotenv
load_dotenv()

invoke_tps_lambda_name = os.environ["INVOKE_TPS_SM_NAME"]

boto3_session = boto3.Session()
lambda_client = boto3_session.client('lambda')

#### -----------------------
# How to onboard a customer
## Customer Information

In [ ]:
customer_id = "democustomer"
customer_name = "DemoCustomer"

# (Optional) If multi-credential secret configured, Auth Id to use for API calls made to this instance.
# If you did not configure your secret for multiple credentials, leave as None.
auth_id = None

## AMC Instance Information

In [ ]:
data_upload_account_id = "<data upload aws account id>"
bucket_name = "<amc s3 bucket name>"
instance_id = "<amc instance id>"
amazon_ads_advertiser_id="<amazon ads advertiser id>"
amazon_ads_marketplace_id="<amazon ads marketplace id>"

## Select *one* of the patterns below:

### 1. Bucket does not exist
Choose this option if your AMC Instance S3 Bucket does not exist. The application will attempt to deploy it for you in the target region.

In [ ]:
bucket_region = '<region to deploy the s3 bucket>'
bucket_exists = "false"

onboard_customer_request = {
    "customer_details" : {
        "auth_id": auth_id,
        "customer_id": customer_id,
        "customer_name": customer_name,
        "bucket_region": bucket_region,
        "bucket_exists": bucket_exists,
        "amc": {
            "aws_orange_account_id": data_upload_account_id,
            "bucket_name": bucket_name, 
            "instance_id": instance_id,
            "amazon_ads_advertiser_id": amazon_ads_advertiser_id,
            "amazon_ads_marketplace_id": amazon_ads_marketplace_id,
        }
    }
}

### 2. Bucket exists in the current AWS account
Choose this option if your AMC Instance S3 Bucket already exists and is in the same account as this application. This may be from having previously onboarded this customer through AMC Insights already or from using AMC prior to installation of the application.

In [ ]:
bucket_region = '<aws region of the s3 bucket>'
bucket_exists = "true"

onboard_customer_request = {
    "customer_details" : {
        "auth_id": auth_id,
        "customer_id": customer_id,
        "customer_name": customer_name,
        "bucket_region": bucket_region,
        "bucket_exists": bucket_exists,
        "amc": {
            "aws_orange_account_id": data_upload_account_id,
            "bucket_name": bucket_name,
            "instance_id": instance_id,
            "amazon_ads_advertiser_id": amazon_ads_advertiser_id,
            "amazon_ads_marketplace_id": amazon_ads_marketplace_id,
        }
    }
}

### 3. Bucket exists in a separate AWS account
Choose this option if your AMC Instance S3 Bucket already exists but in a different account than this application. Refer to the [Implementation Guide](https://docs.aws.amazon.com/solutions/latest/amazon-marketing-cloud-insights-on-aws/steps-to-enable-cross-account-data-lake-integration.html) for additional steps needed to be taken to enable cross-account integration.

In [ ]:
bucket_account = '<aws account id of the s3 bucket>'

onboard_customer_request = {
    "customer_details" : {
        "auth_id": auth_id,
        "customer_id": customer_id,
        "customer_name": customer_name,
        "bucket_account": bucket_account,
        "amc": {
            "aws_orange_account_id": data_upload_account_id,
            "bucket_name": bucket_name,
            "instance_id": instance_id,
            "amazon_ads_advertiser_id": amazon_ads_advertiser_id,
            "amazon_ads_marketplace_id": amazon_ads_marketplace_id,
        }
    }
}

## Run the below cell to setup and verify the customer configuration

In [ ]:
print(json.dumps(onboard_customer_request, indent=4))

### Run the below cell to submit the customer configuration. 
#### Before submitting verify the configuration by running the above cell

In [ ]:
response = lambda_client.invoke(
    FunctionName=invoke_tps_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(onboard_customer_request).encode('UTF-8'),
)

response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

#### -----------------------
# Onboard multiple customers
The code below demonstrates how you can pass multiple customer configurations in a single list. Each customer will be onboarded separately.

In [ ]:
customer_list = [
    # same-account existing bucket
    {
        "customer_details" : {
            "customer_id": "customer_1",
            "customer_name": "customer_name_1",
            "bucket_region": "us-east-1",
            "bucket_exists": "true",
            "amc": {
                "aws_orange_account_id": "1111111111",
                "bucket_name": "amc-customer-1",
                "instance_id": "333333333",
                "amazon_ads_advertiser_id": "ENTITYASDFGHHJK",
                "amazon_ads_marketplace_id": "QWERTYUOPN",
            }
        }
    },
    # same-account nonexisting bucket
    {
        "customer_details" : {
            "customer_id": "customer_2",
            "customer_name": "customer_name_2",
            "bucket_region": "eu-west-1",
            "bucket_exists": "false",
            "amc": {
                "aws_orange_account_id": "9999999999",
                "bucket_name": "amc-customer-2",
                "instance_id": "333333333",
                "amazon_ads_advertiser_id": "ENTITYASDFGHHJK",
                "amazon_ads_marketplace_id": "QWERTYUOPN",
            }
        }
    },
    # cross-account existing bucket
    {
        "customer_details" : {
            "customer_id": "customer_3",
            "customer_name": "customer_name_3",
            "bucket_account": "44444444444",
            "amc": {
                "aws_orange_account_id": "9999999999",
                "bucket_name": "amc-customer-3",
                "instance_id": "333333333",
                "amazon_ads_advertiser_id": "ENTITYASDFGHHJK",
                "amazon_ads_marketplace_id": "QWERTYUOPN",
            }
        }
    }
]

In [ ]:
for onboard_customer_request in customer_list:
    time.sleep(10) # Customers should not be onboarded at the same time. Add a waiting time before onboarding the next customer.
    customer_id = onboard_customer_request['customer_details']['customer_id']
    try:
        response = lambda_client.invoke(
            FunctionName=invoke_tps_lambda_name,
            InvocationType='RequestResponse',
            LogType='Tail',
            Payload=json.dumps(onboard_customer_request).encode('UTF-8'),
        )

        response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
        if response_code in range(200, 204):
            payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
            payload_json = json.loads(payload_json)
            executionArn = payload_json.get('executionArn', '')
            sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
                "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
            execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
            print(f"Execution URL for customer {customer_id}: {execution_URL}")
    except Exception as e:
        print(f'Unable to submit onboard request for customer {customer_id}. Please wait and retry again.')

----
Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0